# Gather & Clean Data
## Step 1: Import Modules & API keys

In [1]:
# Import modules
import pandas as pd
import json
import requests
import pprint
import os

# Career One Stop API credentials
from config import token
from config import userId

# College Scorecard API key
from config import api_key

## Step 2: Get list of occupations
Source: https://www.onetonline.org/find/career?c=11&g=Go

In [2]:
# Pull occupations from csv file
file_path = os.path.join('Resources','Information_Technology.csv')
o_net_df =  pd.read_csv(file_path, encoding="ISO-8859-1")

## Step 3: Get Occupation Wages & Education
Source: https://www.careeronestop.org/Developers/WebAPI/web-api.aspx  
#### Education Level Key:
1. Less than high school diploma
2. High school diploma or equivalent
3. Some college, no degree
4. Associate's degree
5. Bachelor's degree
6. Master's degree
7. Doctoral or professional degree

In [3]:
# Configure API Header & Location param
headers = {'Authorization': f'Bearer {token}'}
location = 'FL'

# Create variables used within loop
occupation_df = pd.DataFrame()
i = 0

# Loop through each IT occupation from step 2 and retrieve occupation details.
for code in o_net_df['Code']:

    # API call to retrieve occupation details
    keyword = o_net_df['Code'][i]
    params = {'wages': 'true'
             ,'training': 'true'
             }
    api_url = f'https://api.careeronestop.org/v1/occupation/{userId}/{keyword}/{location}?'
    response = requests.get(api_url, headers=headers, params=params).json()
 
    # Loop through wage rates & assign the annual & hourly valiables accordingly
    annual = 0
    hourly = 0
    for row in response['OccupationDetail'][0]['Wages']['StateWagesList']:
        if row['RateType'] == 'Annual':
            annual = row
        elif row['RateType'] == 'Hourly':
            hourly = row
    
    # Loop through education levels & assign valiables accordingly
    EducationLevel_1 = 0
    EducationLevel_2 = 0
    EducationLevel_3 = 0
    EducationLevel_4 = 0
    EducationLevel_5 = 0
    EducationLevel_6 = 0
    EducationLevel_7 = 0

    for row in response['OccupationDetail'][0]['EducationTraining']['EducationType']:
        if row['EducationLevel'] == 'Less than high school diploma':
            EducationLevel_1 = row['Value']
        elif row['EducationLevel'] == 'High school diploma or equivalent':
            EducationLevel_2 = row['Value']
        elif row['EducationLevel'] == 'Some college, no degree':
            EducationLevel_3 = row['Value']
        elif row['EducationLevel'] == "Associate's degree":
            EducationLevel_4 = row['Value']
        elif row['EducationLevel'] == "Bachelor's degree":
            EducationLevel_5 = row['Value']
        elif row['EducationLevel'] == "Master's degree":
            EducationLevel_6 = row['Value']
        elif row['EducationLevel'] == 'Doctoral or professional degree':
            EducationLevel_7 = row['Value']
    
    # Build DataFrame with this instance
    df = pd.DataFrame(
            {'Occupation':[o_net_df['Occupation'][i]]
            ,'Career Pathway':o_net_df['Career Pathway'][i]
            ,'OnetTitle':[response['OccupationDetail'][0]['OnetTitle']]
            ,'OnetDescription':[response['OccupationDetail'][0]['OnetDescription']]
            ,'Annual Wage - Pct10':[annual['Pct10']]
            ,'Annual Wage - Pct25':[annual['Pct25']]
            ,'Annual Wage - Median':[annual['Median']]
            ,'Annual Wage - Pct75':[annual['Pct75']]
            ,'Annual Wage - Pct90':[annual['Pct90']]
            ,'Hourly Wage - Pct10':[hourly['Pct10']]
            ,'Hourly Wage - Pct25':[hourly['Pct25']]
            ,'Hourly Wage - Median':[hourly['Median']]
            ,'Hourly Wage - Pct75':[hourly['Pct75']]
            ,'Hourly Wage - Pct90':[hourly['Pct90']]
            ,'WageYear':[response['OccupationDetail'][0]['Wages']['WageYear']]
            ,'BrightOutlook':[response['OccupationDetail'][0]['BrightOutlook']]
            ,'EducationLevel_1':[EducationLevel_1]
            ,'EducationLevel_2':[EducationLevel_2]
            ,'EducationLevel_3':[EducationLevel_3]
            ,'EducationLevel_4':[EducationLevel_4]
            ,'EducationLevel_5':[EducationLevel_5]
            ,'EducationLevel_6':[EducationLevel_6]
            ,'EducationLevel_7':[EducationLevel_7]
            ,'EducationCode':[response['OccupationDetail'][0]['EducationTraining']['EducationCode']]
            ,'EducationTitle':[response['OccupationDetail'][0]['EducationTraining']['EducationTitle']]
            ,'ExperienceCode':[response['OccupationDetail'][0]['EducationTraining']['ExperienceCode']]
            ,'ExperienceTitle':[response['OccupationDetail'][0]['EducationTraining']['ExperienceTitle']]
            ,'TrainingCode':[response['OccupationDetail'][0]['EducationTraining']['TrainingCode']]
            ,'TrainingTitle':[response['OccupationDetail'][0]['EducationTraining']['TrainingTitle']]
            ,'EducationOccupationTitle':[response['OccupationDetail'][0]['EducationTraining']['OccupationTitle']]
            }, index=[o_net_df['Code'][i]])
    occupation_df = occupation_df.append(df)
    i += 1

occupation_df.head()

,Occupation,Career Pathway,OnetTitle,OnetDescription,Annual Wage - Pct10,Annual Wage - Pct25,Annual Wage - Median,Annual Wage - Pct75,Annual Wage - Pct90,Hourly Wage - Pct10,...,EducationLevel_5,EducationLevel_6,EducationLevel_7,EducationCode,EducationTitle,ExperienceCode,ExperienceTitle,TrainingCode,TrainingTitle,EducationOccupationTitle
15-2051.01,Business Intelligence Analysts,Information Support and Services,Business Intelligence Analysts,Produce financial and market intelligence by q...,44810,54300,64250,98980,138330,21.55,...,37.5,36,14.5,3,Bachelor's degree,3,No work experience,6,No on-the-job training,Data Scientists and Mathematical Science Occup...
15-1299.00,"Computer Occupations, All Other",Information Support and Services,"Computer Occupations, All Other",All computer occupations not listed separately.,31020,45760,71250,98880,123080,14.91,...,43.6,20,2.2,3,Bachelor's degree,3,No work experience,6,No on-the-job training,"Computer Occupations, All Other"
15-1299.08,Computer Systems Engineers/Architects,Information Support and Services,Computer Systems Engineers/Architects,Design and develop solutions to complex applic...,31020,45760,71250,98880,123080,14.91,...,43.6,20,2.2,3,Bachelor's degree,3,No work experience,6,No on-the-job training,"Computer Occupations, All Other"
15-1232.00,Computer User Support Specialists,Information Support and Services,Computer User Support Specialists,Provide technical assistance to computer users...,30040,36570,46970,61660,79350,14.44,...,37,10.7,1,6,"Some college, no degree",3,No work experience,6,No on-the-job training,Computer User Support Specialists
15-1243.01,Data Warehousing Specialists,Information Support and Services,Data Warehousing Specialists,"Design, model, or implement corporate data war...",45430,63700,90230,116170,134290,21.84,...,46.9,25.2,2.7,3,Bachelor's degree,3,No work experience,6,No on-the-job training,Database Administrators and Architects


## Step 4: Pull tuition data for Florida schools
Source: https://collegescorecard.ed.gov/data/
#### Degree Level Key:
1. Undergraduate Certificates or Diplomas
2. Associate’s Degrees
3. Bachelor’s Degrees
4. Post-Baccalaureate Certificates
5. Master’s Degrees
6. Doctoral Degrees
7. First Professional Degrees
8. Graduate / Professional Certificates

In [4]:
# Params
base_url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?'
state = 'school.state=FL&fields=id,school.name,school.state' 
year = 2018
cost_per = f'{year}.cost.attendance.academic_year,{year}.cost.attendance.program_year'
predom_type = 'school.degrees_awarded.predominant_recoded'
response_ls = []
# Loop through pages of API call

for page in range(20):
    
    # Grab the base url
    url = f'{base_url}{state},{cost_per},{predom_type}&api_key={api_key}&page={page}'
    response = requests.get(url).json()
    response_ls.append(response)
response_ls

[{'metadata': {'total': 387, 'page': 0, 'per_page': 20},
  'results': [{'school.name': 'Palm Beach Academy of Health & Beauty',
    'school.state': 'FL',
    'id': 449490,
    'school.degrees_awarded.predominant_recoded': 1,
    '2018.cost.attendance.program_year': 25572,
    '2018.cost.attendance.academic_year': None},
   {'school.name': 'SABER College',
    'school.state': 'FL',
    'id': 449506,
    'school.degrees_awarded.predominant_recoded': 2,
    '2018.cost.attendance.program_year': 19233,
    '2018.cost.attendance.academic_year': None},
   {'school.name': 'Pensacola School of Massage Therapy & Health Careers',
    'school.state': 'FL',
    'id': 449533,
    'school.degrees_awarded.predominant_recoded': 1,
    '2018.cost.attendance.program_year': 21608,
    '2018.cost.attendance.academic_year': None},
   {'school.name': 'Paul Mitchell the School-Tampa',
    'school.state': 'FL',
    'id': 449959,
    'school.degrees_awarded.predominant_recoded': 1,
    '2018.cost.attendance.pro

In [5]:
# Create DataFrame
school_df = pd.DataFrame()
page_num = 0
for page in response_ls: 

# Start of loop to append columns in DataFrame
    for x in response_ls[page_num]['results']:
        df = pd.DataFrame(
            {'School Name': [x['school.name']]
            ,'School State':[x['school.state']]
            ,'Degree Type': [x['school.degrees_awarded.predominant_recoded']]
            ,'Program Cost':[x[f'{year}.cost.attendance.program_year']]
            ,'Annual Cost':[x[f'{year}.cost.attendance.academic_year']]
            }, index = [x['id']])
        school_df = school_df.append(df)
    page_num += 1
# Display data
# len(school_df)
# school_df['School Name'].nunique()
school_df.head()

,School Name,School State,Degree Type,Program Cost,Annual Cost
449490,Palm Beach Academy of Health & Beauty,FL,1,25572,None
449506,SABER College,FL,2,19233,None
449533,Pensacola School of Massage Therapy & Health C...,FL,1,21608,None
449959,Paul Mitchell the School-Tampa,FL,1,21596,None
40884402,Florida National University Training Center,FL,2,None,None


In [6]:
# Add total colum to DataFrame
school_df['Total Cost'] = ''

# Iterate through rows by degree type
for index, row in school_df.iterrows():
    if row['Annual Cost'] == None:
        row['Total Cost'] = row['Program Cost']
    # (3) = Bach Degree
    elif row['Degree Type'] == 3:
        row['Total Cost'] = row['Annual Cost']*4
    # (2) = A.A Degree
    elif row['Degree Type'] == 2:
        row['Total Cost'] = row['Annual Cost']*2
    # If not an A.A or Bach, we are assumming it is a 1 year certificate
    else:
        row['Total Cost'] = row['Annual Cost']
school_df.head()

,School Name,School State,Degree Type,Program Cost,Annual Cost,Total Cost
449490,Palm Beach Academy of Health & Beauty,FL,1,25572,None,25572
449506,SABER College,FL,2,19233,None,19233
449533,Pensacola School of Massage Therapy & Health C...,FL,1,21608,None,21608
449959,Paul Mitchell the School-Tampa,FL,1,21596,None,21596
40884402,Florida National University Training Center,FL,2,None,None,None


## Step 5: Clean School Data

In [7]:
# Removing schools that are not needed for data
to_remove = ['Beauty', 'Therapy', 'Med', 'Salon', 'Spa', 
             'Pet', 'Welding', 'Cosmetology', 'Massage', 'Nursing', 
             'Barber', 'Art', 'Airline', 'Law', 'Paul Mitchell', 
             'Neurosomatic', 'Personal Training', 'Seminary', 'Aveda', 'Mechanic', 
             'Bible', 'Hair', 'Aeronautical', 'Ultrasound', 'Aviation', 
             'Health', 'Midwifery', 'Aerospace', 'Professional Hands', 'Rabbinical']
clean_schools = school_df
for word in to_remove:
    clean_schools = clean_schools[clean_schools['School Name'].str.contains(word) == False]

# Dropping null rows
clean_schools = clean_schools.dropna(subset=['Total Cost', 'Degree Type'])

## Step 6: Write Clean Data to CSV Files

In [8]:
occupation_df.to_csv("Clean_Data/CareerOneStop.csv", index=False, header=True)
clean_schools.to_csv('Clean_schools.csv', header=True)